In [2]:
import numpy as np
import pandas as pd

In [3]:
order_new = pd.read_csv("order_new.csv")
order_product_new = pd.read_csv("order_product_new.csv")


In [22]:
order_product_prior = pd.read_csv("order_product_prior.csv")
up_feature = pd.read_csv("up_feature.csv")
user_feature_1 = pd.read_csv("user_feature_1.csv")
user_feature_2 = pd.read_csv("user_feature_2.csv")

KeyboardInterrupt: 

In [25]:
order_new

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2539329,1,1,2,8,NaN,Prior
1,2398795,1,2,3,7,15.0,Prior
2,473747,1,3,3,12,21.0,Prior
3,2254736,1,4,4,7,29.0,Prior
4,431534,1,5,4,15,28.0,Prior
...,...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0,Prior
3421079,1854736,206209,11,4,10,30.0,Prior
3421080,626363,206209,12,1,12,18.0,Prior
3421081,2977660,206209,13,1,12,7.0,Prior


In [26]:
order_product_new

,order_id,product_id,add_to_cart_order,reorder
0,2,33120,1,0.0
1,2,28985,2,0.0
2,2,9327,3,0.0
3,2,45918,4,0.0
4,2,30035,5,0.0
...,...,...,...,...
33819101,3421063,14233,3,1.0
33819102,3421063,35548,4,1.0
33819103,3421070,35951,1,1.0
33819104,3421070,16953,2,1.0


In [27]:
# join orders_new and order_product_new together
join = pd.merge(order_product_new, order_new, on = 'order_id', how = 'left' )
join

,order_id,product_id,add_to_cart_order,reorder,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2,33120,1,0.0,202279,3,5,9,8.0,Prior
1,2,28985,2,0.0,202279,3,5,9,8.0,Prior
2,2,9327,3,0.0,202279,3,5,9,8.0,Prior
3,2,45918,4,0.0,202279,3,5,9,8.0,Prior
4,2,30035,5,0.0,202279,3,5,9,8.0,Prior
...,...,...,...,...,...,...,...,...,...,...
33819101,3421063,14233,3,1.0,169679,30,0,10,4.0,Train
33819102,3421063,35548,4,1.0,169679,30,0,10,4.0,Train
33819103,3421070,35951,1,1.0,139822,15,6,10,8.0,Train
33819104,3421070,16953,2,1.0,139822,15,6,10,8.0,Train


In [28]:
order_product_prior = join[join['eval_set'] == 'Prior']
order_product_prior.shape

(32434489, 10)

In [29]:
order_product_prior.head()

,order_id,product_id,add_to_cart_order,reorder,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2,33120,1,0.0,202279,3,5,9,8.0,Prior
1,2,28985,2,0.0,202279,3,5,9,8.0,Prior
2,2,9327,3,0.0,202279,3,5,9,8.0,Prior
3,2,45918,4,0.0,202279,3,5,9,8.0,Prior
4,2,30035,5,0.0,202279,3,5,9,8.0,Prior


In [12]:
order_product_prior.to_csv('order_product_prior.csv',index=False)

In [30]:
# Based on table orders_new, for each user, calculate the max order_number, the sum of 
#days_since_prior_order and the average of days_since_prior_order. 
df1 = order_new[order_new['eval_set'] == 'Prior']

user_id = df1['user_id'].unique()
user_orders = df1.groupby(['user_id'])['order_number'].max()
sum_days_since_prior_order = df1.groupby(['user_id'])['days_since_prior_order'].sum()
avg_days_since_prior_order = df1.groupby(['user_id'])['days_since_prior_order'].mean()

In [31]:
user_feature_1 = pd.DataFrame({
    'user_orders': user_orders,
    'sum_days_since_prior_order': sum_days_since_prior_order,
    'avg_days_since_prior_order': avg_days_since_prior_order
})
user_feature_1 = user_feature_1.reset_index()
user_feature_1.head(10)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order
0,1,10,176.0,19.555556
1,2,14,198.0,15.230769
2,3,12,133.0,12.090909
3,4,5,55.0,13.750000
4,5,4,40.0,13.333333
5,6,3,18.0,9.000000
6,7,20,203.0,10.684211
7,8,3,60.0,30.000000
8,9,3,36.0,18.000000
9,10,5,79.0,19.750000


In [16]:
user_feature_1.to_csv('user_feature_1.csv',index=False)

In [32]:
#for each user calculate the total number of products, total number of distinct products, 
#and user reorder ratio(number of reordered = 1 divided by number of order_number > 1)
df = order_product_prior[order_product_prior['order_number'] > 1]

total_products = order_product_prior.groupby('user_id')['product_id'].count()
distinct_products = order_product_prior.groupby('user_id')['product_id'].nunique()
reordered_sum = order_product_prior.groupby('user_id')['reorder'].sum()
reordered_count = df.groupby('user_id')['reorder'].count()

In [33]:
df

,order_id,product_id,add_to_cart_order,reorder,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2,33120,1,0.0,202279,3,5,9,8.0,Prior
1,2,28985,2,0.0,202279,3,5,9,8.0,Prior
2,2,9327,3,0.0,202279,3,5,9,8.0,Prior
3,2,45918,4,0.0,202279,3,5,9,8.0,Prior
4,2,30035,5,0.0,202279,3,5,9,8.0,Prior
...,...,...,...,...,...,...,...,...,...,...
32434484,3421083,39678,6,1.0,25247,24,2,6,21.0,Prior
32434485,3421083,11352,7,1.0,25247,24,2,6,21.0,Prior
32434486,3421083,4600,8,1.0,25247,24,2,6,21.0,Prior
32434487,3421083,24852,9,1.0,25247,24,2,6,21.0,Prior


In [34]:
user_feature_2 = pd.DataFrame({
    'total_products': total_products,
    'distinct_products': distinct_products,
    'reordered_ratio': reordered_sum/reordered_count
})
user_feature_2.head(10)

,total_products,distinct_products,reordered_ratio
user_id,,,
1,59,18,0.759259
2,195,102,0.576923
3,88,33,0.666667
4,18,17,0.642857
5,37,23,0.961538
6,14,12,0.400000
7,206,68,0.680412
8,49,36,1.000000
9,76,58,1.000000


In [19]:
user_feature_2.to_csv('user_feature_2.csv',index=False)

In [35]:
#for each user and product, calculate the total number of orders, minimum order_number,
#maximum order_number and average add_to_cart_order.
m = order_product_prior.groupby(['user_id','product_id'])

In [36]:
up_feature = pd.DataFrame({
    'up_orders': m['order_number'].count(),
    'up_first_order': m['order_number'].min(),
    'up_last_order': m['order_number'].max(),
    'up_average_cart_position': m['add_to_cart_order'].mean()
})

In [37]:
up_feature = up_feature.reset_index()
up_feature.head(10)

,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position
0,1,196,10,1,10,1.400000
1,1,10258,9,2,10,3.333333
2,1,10326,1,5,5,5.000000
3,1,12427,10,1,10,3.300000
4,1,13032,3,2,10,6.333333
5,1,13176,2,2,5,6.000000
6,1,14084,1,1,1,2.000000
7,1,17122,1,5,5,6.000000
8,1,25133,8,3,10,4.000000
9,1,26088,2,1,2,4.500000


In [23]:
up_feature.to_csv('up_feature.csv',index=False)

In [38]:
#calculate the sequence of product purchase for each user (e.g. if a user bought egg product 
#in the first oder and second order, the value should be 1 and 2 respectively) and name it 
#product_seq_time.
order_product_prior['product_seq_time'] = order_product_prior.groupby(['user_id','product_id'])['order_number'].rank()



/Users/apple/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [39]:
m = order_product_prior.groupby('product_id')

In [40]:
#for each product,  calculate the count, sum of reordered, count of product_seq_time = 1
#(hint: Sum(CASE WHEN product_seq_time = 1 THEN 1 ELSE 0 END)) and count of product_seq_time = 2.
prd_feature = pd.DataFrame({
    'prod_orders': m['order_number'].count(),
    'prod_reorders': m['reorder'].sum(),
    'prod_first_orders': order_product_prior[order_product_prior['product_seq_time'] == 1].groupby('product_id')['product_seq_time'].sum(),
    'prod_second_orders': order_product_prior[order_product_prior['product_seq_time'] == 2].groupby('product_id')['product_seq_time'].count()
})

In [41]:
prd_feature = prd_feature.reset_index()
prd_feature.head(10)

,product_id,prod_orders,prod_reorders,prod_first_orders,prod_second_orders
0,1,1852,1066.0,716.0,276.0
1,2,90,51.0,78.0,8.0
2,3,277,167.0,74.0,36.0
3,4,329,192.0,182.0,64.0
4,5,15,8.0,6.0,4.0
5,6,8,3.0,5.0,2.0
6,7,30,22.0,18.0,6.0
7,8,165,88.0,82.0,30.0
8,9,156,91.0,74.0,31.0
9,10,2572,1481.0,1268.0,399.0


In [8]:
prd_feature.to_csv('prd_feature.csv',index=False)

In [43]:
data = pd.merge(user_feature_1, user_feature_2, on = 'user_id', how = 'left')
data = pd.merge(data, up_feature, on = 'user_id', how = 'left')
data = pd.merge(data, prd_feature, on = 'product_id', how = 'left')

In [44]:
data.head(10)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorders,prod_first_orders,prod_second_orders
0,1,10,176.0,19.555556,59,18,0.759259,196,10,1,10,1.400000,35791,20502.0,8000.0,4660.0
1,1,10,176.0,19.555556,59,18,0.759259,10258,9,2,10,3.333333,1946,1092.0,557.0,308.0
2,1,10,176.0,19.555556,59,18,0.759259,10326,1,5,5,5.000000,5526,3114.0,1923.0,1003.0
3,1,10,176.0,19.555556,59,18,0.759259,12427,10,1,10,3.300000,6476,3662.0,1679.0,889.0
4,1,10,176.0,19.555556,59,18,0.759259,13032,3,2,10,6.333333,3751,2149.0,1286.0,617.0
5,1,10,176.0,19.555556,59,18,0.759259,13176,2,2,5,6.000000,379450,217064.0,63537.0,45231.0
6,1,10,176.0,19.555556,59,18,0.759259,14084,1,1,1,2.000000,15935,9029.0,3012.0,1895.0
7,1,10,176.0,19.555556,59,18,0.759259,17122,1,5,5,6.000000,13880,8097.0,4503.0,2375.0
8,1,10,176.0,19.555556,59,18,0.759259,25133,8,3,10,4.000000,6196,3509.0,1610.0,935.0
9,1,10,176.0,19.555556,59,18,0.759259,26088,2,1,2,4.500000,2523,1417.0,1163.0,471.0


In [45]:
data['prod_reorder_probability'] = data.prod_second_orders / data.prod_first_orders
data['prod_reorder_times'] = 1 + data.prod_reorders / data.prod_first_orders
data['prod_reorder_ratio'] = data.prod_reorders / data.prod_orders
data.drop(['prod_reorders', 'prod_first_orders', 'prod_second_orders'], axis=1, inplace=True)
data['user_average_basket'] = data.total_products / data.user_orders
data['up_order_rate'] = data.up_orders / data.user_orders
data['up_orders_since_last_order'] = data.user_orders - data.up_last_order
data['up_order_rate_since_first_order'] = data.up_orders / (data.user_orders - data.up_first_order + 1)

In [46]:
prd = data['product_id']
data.drop(labels=['product_id'], axis=1,inplace = True)
data.insert(1, 'product_id', prd)
data.head(10)

,user_id,product_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order
0,1,196,10,176.0,19.555556,59,18,0.759259,10,1,10,1.400000,35791,0.582500,3.562750,0.572826,5.9,1.0,0,1.000000
1,1,10258,10,176.0,19.555556,59,18,0.759259,9,2,10,3.333333,1946,0.552962,2.960503,0.561151,5.9,0.9,0,1.000000
2,1,10326,10,176.0,19.555556,59,18,0.759259,1,5,5,5.000000,5526,0.521581,2.619345,0.563518,5.9,0.1,5,0.166667
3,1,12427,10,176.0,19.555556,59,18,0.759259,10,1,10,3.300000,6476,0.529482,3.181060,0.565473,5.9,1.0,0,1.000000
4,1,13032,10,176.0,19.555556,59,18,0.759259,3,2,10,6.333333,3751,0.479782,2.671073,0.572914,5.9,0.3,0,0.333333
5,1,13176,10,176.0,19.555556,59,18,0.759259,2,2,5,6.000000,379450,0.711884,4.416340,0.572049,5.9,0.2,5,0.222222
6,1,14084,10,176.0,19.555556,59,18,0.759259,1,1,1,2.000000,15935,0.629150,3.997676,0.566614,5.9,0.1,9,0.100000
7,1,17122,10,176.0,19.555556,59,18,0.759259,1,5,5,6.000000,13880,0.527426,2.798135,0.583357,5.9,0.1,5,0.166667
8,1,25133,10,176.0,19.555556,59,18,0.759259,8,3,10,4.000000,6196,0.580745,3.179503,0.566333,5.9,0.8,0,1.000000
9,1,26088,10,176.0,19.555556,59,18,0.759259,2,1,2,4.500000,2523,0.404987,2.218401,0.561633,5.9,0.2,8,0.200000


In [47]:
data.to_csv('data.csv',index=False)

In [4]:
data = pd.read_csv("data.csv")
data

,user_id,product_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order
0,1,196,10,176.0,19.555556,59,18,0.759259,10,1,10,1.400000,35791,0.582500,3.562750,0.572826,5.900000,1.000000,0,1.000000
1,1,10258,10,176.0,19.555556,59,18,0.759259,9,2,10,3.333333,1946,0.552962,2.960503,0.561151,5.900000,0.900000,0,1.000000
2,1,10326,10,176.0,19.555556,59,18,0.759259,1,5,5,5.000000,5526,0.521581,2.619345,0.563518,5.900000,0.100000,5,0.166667
3,1,12427,10,176.0,19.555556,59,18,0.759259,10,1,10,3.300000,6476,0.529482,3.181060,0.565473,5.900000,1.000000,0,1.000000
4,1,13032,10,176.0,19.555556,59,18,0.759259,3,2,10,6.333333,3751,0.479782,2.671073,0.572914,5.900000,0.300000,0,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13307948,206209,43961,13,210.0,17.500000,129,68,0.715517,3,4,12,8.000000,55371,0.514642,2.552628,0.573567,9.923077,0.230769,1,0.300000
13307949,206209,44325,13,210.0,17.500000,129,68,0.715517,1,7,7,8.000000,3485,0.289890,1.931960,0.558106,9.923077,0.076923,6,0.142857
13307950,206209,48370,13,210.0,17.500000,129,68,0.715517,1,11,11,8.000000,3934,0.592561,2.900254,0.571429,9.923077,0.076923,2,0.333333
13307951,206209,48697,13,210.0,17.500000,129,68,0.715517,1,7,7,6.000000,9783,0.260662,1.874443,0.561689,9.923077,0.076923,6,0.142857


In [5]:
order_new_s = order_new[['user_id','eval_set']]

In [5]:
data_new = data.sample(frac = 0.001,replace=False,random_state=1)
data_new

,user_id,product_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order
5137560,79375,38230,9,135.0,16.875000,58,37,0.725490,2,8,9,6.5,96,0.243243,1.716216,0.552083,6.444444,0.222222,0,1.000000
12489928,193492,44525,4,64.0,21.333333,79,58,1.000000,1,4,4,12.0,100,0.141026,1.833333,0.650000,19.750000,0.250000,0,1.000000
1821163,28365,12254,19,275.0,15.277778,138,89,0.449612,1,4,4,22.0,5629,0.385714,2.031169,0.564221,7.263158,0.052632,15,0.062500
2773203,43099,5194,4,69.0,23.000000,52,43,1.368421,1,4,4,18.0,13372,0.433924,2.405685,0.569548,13.000000,0.250000,0,1.000000
1185921,18502,22699,3,20.0,10.000000,27,24,0.588235,1,3,3,8.0,2902,0.403636,2.214545,0.575465,9.000000,0.333333,0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13284241,205833,33929,38,357.0,9.648649,481,127,0.475269,1,3,3,21.0,2885,0.333083,2.254887,0.578510,12.657895,0.026316,35,0.027778
12459123,193031,41319,13,183.0,15.250000,101,54,0.538462,2,1,5,10.5,6671,0.293649,2.013371,0.590766,7.769231,0.153846,8,0.153846
3935030,61001,25146,4,60.0,20.000000,28,18,0.812500,1,2,2,1.0,24918,0.454087,2.572024,0.575046,7.000000,0.250000,2,0.333333
782425,12300,34019,38,356.0,9.621622,1349,376,0.632622,1,26,26,38.0,765,0.235178,1.859684,0.568627,35.500000,0.026316,12,0.076923


In [ ]:
#这里服务器挂掉了
new_dataframe = pd.merge(data,order_new_s, on = 'user_id', how = 'left' )
#new_dataframe

In [6]:
new_dataframe_s = pd.merge(order_new_s, data_new, on = 'user_id', how = 'left' )
new_dataframe_s

,user_id,eval_set,product_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,up_orders,...,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order
0,1,Prior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Prior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Prior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Prior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Prior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3445061,206209,Prior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3445062,206209,Prior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3445063,206209,Prior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3445064,206209,Prior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
Train1 = new_dataframe_s[new_dataframe_s.eval_set == 'Train']
Train1

,user_id,eval_set,product_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,up_orders,...,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order
10,1,Train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2,Train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,5,Train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,7,Train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,8,Train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444821,206199,Train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3444845,206200,Train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3444907,206203,Train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3444916,206205,Train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Test = new_dataframe[new_dataframe.eval_set == 'Test']
Test

In [ ]:
Target = pd.merge(order_product_new, train1, on = 'product_id', how = 'left')
Target.head()

In [ ]:
Target['reorder'].value_counts()

In [ ]:
Train=Target.fillna(value=0)

In [ ]:
data_xgb = data.sample(frac = 0.001,replace = False,random_state=1)